# Korean Lexical analyzer with Recursion

Reference:
* [1] [한국어 형태소 분석기와 품사 태거 구현](https://github.com/gritmind/morph_and_pos_analyzer_korean)
* [2] [자연어처리 - 2. Morphological Analysis (Korean)](https://github.com/gritmind/review-media/blob/master/class/natural-language-processing-chungnam/README.md)

In [1]:
####################################################################################
""" Load Vocabuary """
####################################################################################
vocab = [
    '고전',
    '고전주의',
    '신고',
    '신',
    '전주',
    '주의',
    '의정',
    '정신',
    '을'
    
]

In [2]:
import sys, os

## Global Variable
hit_set = [] # 2d-array, 모든 종류의 단어조합(=thread)를 저장하기 위해.
n_thr = -1 # unique thread index

def recurs(sub_list, thread, depth):
    ## search strategy - recursive function
    global n_thr 
    global hit_set
    depth += 1 # `recurs` 함수가 불린 횟수 (=tree depth level)
    cur_depth = depth # 현재 depth 레벨 저장 - 가지치기용.
    temp_str = ''
    tnk = False # 기존 것과 새로운 thread를 구분하기 위한 토큰.
    #print(idx, length)
    #print(n_thr)

    for i, x in enumerate(sub_list):
        temp_str += x
        if temp_str in vocab:
            #thread.append(temp_str)
            #print(thread)
            if tnk == False: # 기존의 thread를 계속 이어감.
                #print('---',temp_str)
                #print('cur_depth:', cur_depth)
                thread.append(temp_str)
                hit_set[n_thr].append(temp_str)
                
            elif tnk == True: # 새로운 thread를 생성함.
                n_thr += 1
                #print('===>',temp_str)
                #print('cur_depth:', cur_depth)
                thread = thread[:cur_depth] # 새로운 thread 생성 - 현재 depth 정보를 참고하여 기존 thread에서 가지치기 실시. 
                thread.append(temp_str)
                temp_thread = thread[:] # 주소 공유를 방지하기 위해 값을 할당.
                hit_set.insert(n_thr, temp_thread)
            
            ## Continue searching recursively
            recurs(sub_list[i+1:], thread, depth)
            tnk = True # 지금 현재의 `recurs` 함수는 이제 새로운 thread이다.

In [3]:
def main(input_):
    global n_thr
    global hit_set

    ###################################
    """ Main Thread """
    ###################################
    temp_str = ''
    thread = []
    depth = 0
    input_list = list(input_)
    length = len(input_list)
    
    for i, x in enumerate(input_list):
        temp_str += x
        if temp_str in vocab: # 여기서는 항상 새로운 thread가 생성된다.
            n_thr += 1
            #print(temp_str)
            #thread.append(temp_str)
            hit_set.insert(n_thr, [temp_str])
            ##################################
            """ Recursion Start """
            recurs(input_list[i+1:], [temp_str], depth)	

    for each in hit_set:
        sum_len = sum([len(list(x)) for x in each])
        if length == sum_len:
            print('+'.join(each))	

In [4]:
main("신고전주의정신을")

신+고전+주의+정신+을
신+고전주의+정신+을
신고+전주+의정+신+을


Above result is the same as [2].

<요약>
* 복합 명사를 음절 단위로 분할하여 리스트로 저장합니다.
* 오른쪽 방향으로 음절을 하나씩 병합해가면서 사전 단어들과 매칭을 확인합니다. 
* 재귀 함수를 통해서 연이은 단어 매칭을 실시하고 가능한 단어 조합을 모두 저장합니다.
* 찾아낸 모든 단어 조합 중에서 입력 음절 개수를 기준으로 정답을 선택합니다.